In [ ]:
import numpy as np

class Scheduler:
    """ 使用仿射变化生成各类数据流 """
    def __init__(self):
        # self.data_flow = 0
        pass
    
    def schedule(self, A, B, dataflow):
        ''' B已转置 '''
        ''' 目前不支持分块 '''
        if (dataflow == 'OS'):
            A_new, B_new = self.OS(A, B)
        elif (dataflow == 'WS'):
            A_new, B_new = self.WS(A, B)
        elif (dataflow == 'RS'):
            A_new, B_new = self.RS(A, B)
        elif (dataflow == 'IS'): 
            A_new, B_new = self.IS(A, B)
        else: 
            exit()
        
        return A_new, B_new

    def affine(self, matrix, I_new, J_new, i_new, j_new):
        affined_matrix = np.zeros((I_new, J_new), dtype=matrix.dtype)  # 初始化新矩阵
        affined_matrix[i_new, j_new] = matrix
        return affined_matrix
    
    def OS(self, A, B):
        (I_a, J_a) = A.shape
        (I_b, J_b) = B.shape

        (i_a, j_a) = np.indices(A.shape)
        (i_b, j_b) = np.indices(B.shape)

        # A_new = affine(A, A.shape[0]+A.shape[1]-1, A.shape[0], np.indices(A.shape)[0]+np.indices(A.shape)[1], np.indices(A.shape)[0])
        # B_new = affine(B, B.shape[0]+A.shape[1]-1, B.shape[0], np.indices(B.shape)[0]+np.indices(B.shape)[1], np.indices(B.shape)[0])
        A_new = self.affine(A, I_a+J_a-1, I_a, i_a+j_a, i_a)
        B_new = self.affine(B, I_b+J_b-1, I_b, i_b+j_b, i_b)
        
        return A_new, B_new
    

    def WS(self, A, B):
        (I_a, J_a) = A.shape
        (I_b, J_b) = B.shape

        (i_a, j_a) = np.indices(A.shape)
        (i_b, j_b) = np.indices(B.shape)

        # A_new = affine(A, A.shape[0]+A.shape[1]-1, A.shape[0], np.indices(A.shape)[0]+np.indices(A.shape)[1], np.indices(A.shape)[0])
        # B_new = affine(B, B.shape[0]+A.shape[1]-1, B.shape[0], np.indices(B.shape)[0]+np.indices(B.shape)[1], np.indices(B.shape)[0])
        A_new = self.affine(A, I_a+J_a-1, I_a, i_a+j_a, i_a)
        B_new = self.affine(B, I_b+J_b-1, I_b, i_b+j_b, i_b)
        
        return A_new, B_new
    
    def RS(self, A, B):
        pass

    def IS(self, A, B):
        pass


In [ ]:
import numpy as np
import pdb


class PE:
    """
    处理单元（Processing Element）
    每个PE接收来自西边和北边的输入，执行乘法累加操作，并将结果传递到东边和南边
    """
    def __init__(self, x, y):
        self.x = x  # PE在阵列中的行位置
        self.y = y  # PE在阵列中的列位置
        # 输入端口
        self.in_west = 0    # 来自西边的输入（A矩阵的元素）
        self.in_north = 0   # 来自北边的输入（B矩阵的元素）
        # 输出端口
        self.out_east = 0    # 传递到东边的输出
        self.out_south = 0   # 传递到南边的输出
        # 内部状态
        self.accumulator = 0  # 累加器，用于存储乘法累加结果

    def compute(self):
        """ 执行乘法累加操作 """
        product = self.in_west * self.in_north
        self.accumulator += product
        # 将输入元素传递给输出端口，以便下一个PE使用
        self.out_east = self.in_west
        self.out_south = self.in_north


class SystolicArray:
    def __init__(self, systolic_size=(16, 16)):
        self.systolic_row, self.systolic_col = systolic_size
        self.array = [[PE(x, y) for y in range(self.systolic_col)] for x in range(self.systolic_row)]

        # 脉动阵列的结果矩阵
        self.result = np.zeros(systolic_size, dtype=int)

    def step(self, in_west, in_north):
        """ 执行一个时钟周期的计算 """
        # 每个PE执行计算
        for row in self.array:
            for pe in row:
                pe.compute()

        # PE外部输入
        for x in range(self.systolic_row-1):
            self.array[x][0].in_west = in_west[x]   
        for y in range(self.systolic_col-1):
            self.array[0][y].in_north = in_north[y]

        # PE内部流动
        for x in range(self.systolic_row-1):
            for y in range(self.systolic_col-1):
                self.array[x][y+1].in_west = self.array[x][y].out_east
                self.array[x+1][y].in_north = self.array[x][y].out_south

    def get_result(self):
        """ 获取结果矩阵（累加器的值）"""
        for x in range(self.systolic_row):
            for y in range(self.systolic_col):
                self.result[x][y] = self.array[x][y].accumulator
        return self.result
    
    def set_accumulator(self, input_matirx):
        """ 向 SystolicArray 所有 acc 中填入值 """
        for x in range(self.systolic_row):
            for y in range(self.systolic_col):
                self.array[x][y].accumulator = input_matirx[x][y]


def matmul_systolic(A_new, B_new, systolic_array):
    systolic_col = systolic_array.systolic_col
    a_row, a_col = A_new.shape
    b_row, b_col = B_new.shape
    # print(M_plus_K_minus_1, K, K_plus_N_minus_1, N)

    # 总时钟周期数
    total_cycles = np.max([a_row, b_row]) + systolic_col - 1

    for cycle in range(total_cycles):
        in_west = np.pad(A_new[cycle, :], (0, max(0, systolic_col - a_col)), mode='constant', constant_values=0) if cycle < a_row else np.zeros(systolic_col) 
        in_north = np.pad(B_new[cycle, :], (0, max(0, systolic_col - b_col)), mode='constant', constant_values=0) if cycle < b_row else np.zeros(systolic_col)

        # 执行一个时钟周期的计算
        systolic_array.step(in_west, in_north)
        # result = systolic_array.get_result()
        # print(result)
        # pdb.set_trace()

    result = systolic_array.get_result()

    return result



if __name__ == "__main__":
    M, N, K = 4, 8, 16
    np.random.seed(0)  
    A = np.random.randint(1, 5, (M, K))
    B = np.random.randint(1, 5, (K, N))

    systolic_array = SystolicArray(systolic_size=(16, 16))
    scheduler = Scheduler()

    np.set_printoptions(threshold=np.inf)
    print(f"矩阵 A: \n{A}\n")
    print(f"矩阵 B: \n{B}\n")

    A_new, B_new = scheduler.schedule(A, B.T, dataflow = "OS")
    print(f"矩阵 A_new: \n{A_new}\n")
    print(f"矩阵 B_new: \n{B_new}\n")
    
    C_systolic = matmul_systolic(A_new, B_new, systolic_array)
    print(f"结果矩阵 (C = A x B) 通过脉动阵列计算得到:\n{C_systolic}\n")

    C_expected = np.matmul(A, B)
    print(f"C_expected (numpy.matmul): \n{C_expected}\n")

    print("验证通过：脉动阵列的计算结果与numpy.matmul一致。") if np.array_equal(C_systolic[:M, :N], C_expected) else print("验证失败：脉动阵列的计算结果与numpy.matmul不一致。")

    np.set_printoptions(threshold=1000)


矩阵 A: 
[[1 4 2 1 4 4 4 4 2 4 2 3 1 4 3 1]
 [1 1 3 2 3 4 4 3 1 2 2 2 2 1 2 1]
 [4 1 4 2 3 4 4 1 3 4 1 2 4 2 4 4]
 [3 4 1 2 2 2 4 1 4 3 1 4 4 3 4 3]]

矩阵 B: 
[[4 1 3 1 1 1 2 2]
 [3 1 1 2 4 1 2 3]
 [3 4 1 2 2 4 2 2]
 [4 3 4 4 3 3 4 1]
 [3 4 2 1 2 3 1 4]
 [1 3 1 4 4 1 4 1]
 [1 1 1 3 4 1 4 3]
 [4 4 2 2 2 1 2 2]
 [2 4 1 4 2 3 1 2]
 [3 1 3 1 2 4 3 3]
 [2 1 4 2 2 4 1 3]
 [3 4 3 4 4 4 3 2]
 [3 3 4 3 4 4 3 3]
 [4 1 2 3 4 3 2 3]
 [2 1 3 3 4 1 4 3]
 [4 1 1 3 1 3 4 3]]

矩阵 A_new: 
[[1 0 0 0]
 [4 1 0 0]
 [2 1 4 0]
 [1 3 1 3]
 [4 2 4 4]
 [4 3 2 1]
 [4 4 3 2]
 [4 4 4 2]
 [2 3 4 2]
 [4 1 1 4]
 [2 2 3 1]
 [3 2 4 4]
 [1 2 1 3]
 [4 2 2 1]
 [3 1 4 4]
 [1 2 2 4]
 [0 1 4 3]
 [0 0 4 4]
 [0 0 0 3]]

矩阵 B_new: 
[[4 0 0 0 0 0 0 0]
 [3 1 0 0 0 0 0 0]
 [3 1 3 0 0 0 0 0]
 [4 4 1 1 0 0 0 0]
 [3 3 1 2 1 0 0 0]
 [1 4 4 2 4 1 0 0]
 [1 3 2 4 2 1 2 0]
 [4 1 1 1 3 4 2 2]
 [2 4 1 4 2 3 2 3]
 [3 4 2 3 4 3 4 2]
 [2 1 1 2 4 1 1 1]
 [3 1 3 4 2 1 4 4]
 [3 4 4 1 2 1 4 1]
 [4 3 3 2 2 3 2 3]
 [2 1 4 4 2 4 1 2]
 [4 1 2 3 4 4 3 2]
 